In [0]:
%sql
use catalog lucasbruand_catalog;

create schema if not exists vector_search;
use schema vector_search;

In [0]:
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
workspaceUrl = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()

In [0]:
print(DATABRICKS_TOKEN)
print(workspaceUrl)

In [0]:
import requests

# Vector Search index name
index_name = "demo_generator.justin_ward_demo_ward_clinical_assistant.care_ops_pdf_index"
# API endpoint using workspaceUrl variable
url = f"{workspaceUrl}/api/2.0/vector-search/indexes/{index_name}/query"

# Headers
headers = {
    "Authorization": f"Bearer {DATABRICKS_TOKEN}",
    "Content-Type": "application/json;charset=UTF-8",
    "Accept": "application/json, text/plain, */*"
}

# Request payload
payload = {
    "num_results": 1024,
    "columns": ["content", "id"],
    "query_text": "example_query"
}

# Make the POST request
response = requests.post(url, headers=headers, json=payload)

# Display the response
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")

In [0]:
%sql
CREATE OR REPLACE FUNCTION vector_search_rest(
  index_name STRING,
  query_vector ARRAY<DOUBLE>,
  columns ARRAY<STRING>,
  num_results INT,
  workspace_url STRING,
  token STRING
)
RETURNS STRING
LANGUAGE PYTHON
AS $$
import requests
import json

# Get workspace context
try:
    # Build API endpoint
    url = f"{workspace_url}/api/2.0/vector-search/indexes/{index_name}/query"
    
    # Headers
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json;charset=UTF-8",
        "Accept": "application/json, text/plain, */*"
    }
    
    # Request payload
    payload = {
        "num_results": num_results,
        "columns": columns,
        "query_vector": query_vector
    }
    
    # Make the POST request
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        return json.dumps(response.json())
    else:
        return json.dumps({"error": f"Status {response.status_code}", "message": response.text})
except Exception as e:
    return json.dumps({"error": str(e)})
$$

In [0]:
workspace_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
print(workspace_url)
print(token)    

In [0]:
emb = [0.1]*1024
print(len(emb))
print(",".join(str(x) for x in emb))

In [0]:
df = spark.sql(f"""
SELECT vector_search_rest(
  'demo_generator.justin_ward_demo_ward_clinical_assistant.care_ops_pdf_index',
  array({",".join(str(x) for x in emb)}),
  array("content", "id"),
  103,
  workspace_url => '{workspace_url}',
  token => '{token}'
)
""")

In [0]:
display(df)